In [7]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import pickle
import datetime
import types
import pandas.io.sql as sqlio
from psycopg2 import sql


with open("../db_config.json", "r") as db_config:
    db_config = json.load(db_config)

# db 접속정보는 json으로 관리
dbname = db_config['redshift']['NAME']
host = db_config['redshift']['HOST'] 
port = db_config['redshift']['PORT'] 
user = db_config['redshift']['USER']
password= db_config['redshift']['PASSWORD'] 
connect_param = dict({'dbname':dbname, 'host':host, 'port':port, 'user':user, 'password':password})

# 1. 대상 회사 테이블 만들기

In [11]:
with open('/Users/youngjinlim/OneDrive/Coding/BigData_Study/Final_project/Data/Company_list/company_list_df.pkl', 'rb') as f:
    company_list = pickle.load(f)
company_list

,stock_code,company_name,market,bankruptcy,delisted_check,delisted_date,pre_6m,pre_3m,pre_1y
0,008670,굿모닝신한증권,Y,0,1,2005-01-05 00:00:00,2004-07-09 00:00:00,2004-10-07 00:00:00,2004-01-06 00:00:00
1,005600,중앙제지,Y,1,1,2005-01-06 00:00:00,2004-07-10 00:00:00,2004-10-08 00:00:00,2004-01-07 00:00:00
2,035780,그로웰텔레콤,K,0,1,2005-01-11 00:00:00,2004-07-15 00:00:00,2004-10-13 00:00:00,2004-01-12 00:00:00
3,035910,현대멀티캡,K,1,1,2005-01-19 00:00:00,2004-07-23 00:00:00,2004-10-21 00:00:00,2004-01-20 00:00:00
4,009220,그로웰전자,K,0,1,2005-01-19 00:00:00,2004-07-23 00:00:00,2004-10-21 00:00:00,2004-01-20 00:00:00
...,...,...,...,...,...,...,...,...,...
2703,000070,삼양홀딩스,Y,0,0,9999-12-31 00:00:00,2019-07-04 00:00:00,2019-10-02 00:00:00,2018-12-31 00:00:00
2704,000060,메리츠화재,Y,0,0,9999-12-31 00:00:00,2019-07-04 00:00:00,2019-10-02 00:00:00,2018-12-31 00:00:00
2705,000050,경방,Y,0,0,9999-12-31 00:00:00,2019-07-04 00:00:00,2019-10-02 00:00:00,2018-12-31 00:00:00
2706,000040,KR모터스,Y,0,0,9999-12-31 00:00:00,2019-07-04 00:00:00,2019-10-02 00:00:00,2018-12-31 00:00:00


In [ ]:
with psycopg2.connect(**connect_param) as con:
    with con.cursor() as cur:
        # 대상 회사 테이블 만들기
        cur.execute('drop table if exists target_company_list')
        create_companylist_sql = '''
        create table target_company_list(
        stock_code char(6) primary key,
        company_name varchar(50),
        market char(1),
        bankruptcy boolean,
        delisted_check boolean,
        delisted_date date,
        pre_6m date,
        pre_3m date,
        pre_1y date
        );
        '''
        cur.execute(create_companylist_sql)
        con.commit()
        
        # 대상 회사 테이블에 값 넣기
        insert_companyvalue_sql = 'insert into target_company_list values (%s, %s, %s, %s, %s, %s, %s, %s, %s);'
        for each_idx in company_list.index:
            tuple_inserted_listtype = []
            for each_value in company_list.iloc[each_idx].values:
                if str(type(each_value)) == "<class 'numpy.int64'>":
                    tuple_inserted_listtype.append(int(each_value))
                else:
                    tuple_inserted_listtype.append(each_value)
            print(tuple_inserted_listtype)
            cur.execute(insert_companyvalue_sql, tuple(tuple_inserted_listtype))
        
        con.commit() # 테이블 제작완료

In [ ]:
# 올라간 테이블 읽어보기
get_companylist_sql = '''
select * from target_company_list;
'''
dat = sqlio.read_sql_query(get_companylist_sql, con)
dat

# 2. 주식 데이터 테이블 만들기

## 2-1. 주식 데이터 테이블 원본 만들기

In [ ]:
get_header = next(pd.read_csv('stock_data_merged.csv', chunksize=1)).columns

with psycopg2.connect(**connect_param) as con:
    with con.cursor() as cur:
        # 주식 데이터 테이블 만들기
        cur.execute('drop table if exists stock_data_2000_2020_raw')
        create_stocktable_sql = '''
        create table stock_data_2000_2020_raw(
        company_name varchar(50),
        date date,
        stock_code char(6),
        FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code));
        '''
        cur.execute(create_stocktable_sql)
        con.commit()
        
        # 테이블에 열 추가하기
        prefix_add_columns = 'ALTER TABLE stock_data_2000_2020_raw ADD COLUMN'
        for each_column in get_header[3:]:
            add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' double precision'
            cur.execute(add_columns_to_table)
            print(each_column)
        con.commit()

In [ ]:
# 주식 데이터 원본 테이블에 내용 채워넣기
with psycopg2.connect(**connect_param) as con:
    with con.cursor() as cur:
        copy_query_string = """
            copy stock_data_2000_2020_raw
            from 's3://for-bankruptcy-predict/stock_data_merged.csv'
            credentials 'aws_access_key_id={적절한 값 입력};aws_secret_access_key={적절한 값 입력}'
            IGNOREHEADER 1
            CSV;
        """
        cur.execute(copy_query_string)
        con.commit()

## 2-2. 주가 데이터 3년치만 담고 있는 테이블 만들기
* 3개월 전 예측
* 6개월 전 예측
* 1년 전 예측

In [ ]:
for each in ['3m,6m,1y']:
    table_name = 'stock_data_3years_raw_' + each
    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:
            cur.execute('drop table if exists' + table_name)
            create_stocktable_sql = '''
            create table stock_data_3years_raw_3m(
            company_name varchar(50),
            date date,
            stock_code char(6),
            FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code));
            '''
            cur.execute(create_stocktable_sql)

            get_header = next(pd.read_csv('stock_data_merged.csv', chunksize=1)).columns
            # 테이블에 열 추가하기
            prefix_add_columns = 'ALTER TABLE stock_data_3years_raw_3m ADD COLUMN'
            for each_column in get_header[3:]:
                add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' double precision'
                print(add_columns_to_table)
                cur.execute(add_columns_to_table)
            con.commit()

In [2]:
# S3의 csv 파일을 COPY로 Redshift로 옮기는 함수
def copy_csv_from_s3(connect_param, s3_bucket_param=None, table_name=None):
    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:
            credentials = 'aws_access_key_id={적절한 값 입력};aws_secret_access_key={적절한 값 입력}'
            s3_bucket_param = s3_bucket_param

            copy_query = sql.SQL("""
                    copy {table_name}
                    from %(s3_bucket_param)s
                    credentials %(credentials)s
                    IGNOREHEADER 1
                    CSV;
                """).format(table_name = sql.Identifier(table_name))
            
            cur.execute(copy_query, {'s3_bucket_param':s3_bucket_param, 'credentials':credentials})
            con.commit()

In [ ]:
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-predict/stock_data_3years_3m.csv', table_name='stock_data_3years_raw_3m')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-predict/stock_data_3years_6m.csv', table_name='stock_data_3years_raw_6m')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-predict/stock_data_3years_1y.csv', table_name='stock_data_3years_raw_1y')

# 3. 감사보고서 데이터 업로드

In [ ]:
report_df = pd.read_csv('/Users/youngjinlim/OneDrive/Coding/BigData_Study/Final_project/Data/Dart_crawl/dart_audit_report_data.csv', 
                        dtype={'stock_code':'object'}, usecols=['stock_code', 'company_name', 'report_name', 'n_opinion', 'report_date'])
report_df.to_csv('dart_audit_report_data_refined.csv', index=False)
report_df
# n_opinion의 0.0은 0으로, 1.0은 1로 vscode에서 변경하였음

In [ ]:
with psycopg2.connect(**connect_param) as con:
    with con.cursor() as cur:
        # # 감사보고서 테이블 만들기
        cur.execute('drop table if exists dart_audit_report_data')
        create_stocktable_sql = '''
        create table dart_audit_report_data(
        stock_code char(6),
        company_name varchar(50),
        report_name char(4),
        n_opinion boolean,
        report_date date,
        FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code));
        '''
        cur.execute(create_stocktable_sql)
        con.commit()

In [ ]:
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-predict/dart_audit_report_data_refined.csv', table_name='dart_audit_report_data')

# 4. 재무 데이터 테이블 만들기

## 4-1. 재무 데이터 전체 원본 테이블 만들기

In [12]:
df_cache = pd.read_csv('/Users/youngjinlim/OneDrive/Coding/BigData_Study/Final_project/AWS/COPY용 데이터/finance_data_1999_2020_raw.csv', dtype={'stock_code':object}, chunksize=1)
next(df_cache)

,stock_code,company_name,결산월,회계년,총자산(천원),유동자산(천원),현금및현금성자산(천원),재고자산(천원),비유동자산(천원),총부채(천원),...,ROE(당기순이익)(%),총자산회전율(회),매출액증가율(전년동기)(%),매출액증가율(전분기)(%),총자산증가율(전년동기)(%),총자산증가율(전분기)(%),당기순이익증가율(전분기)(%),당기순이익증가율(전년동기)(%),EPS증가율(전년동기)(%),EPS증가율(전분기)(%)
0,008670,굿모닝신한증권,3,2000,1416415964,NaN,319930993,NaN,NaN,786154402,...,12.0,0.29,-42.51,NaN,-25.33,NaN,NaN,-66.18,-66.33,NaN


In [19]:
df_cache = pd.read_csv('/Users/youngjinlim/OneDrive/Coding/BigData_Study/Final_project/AWS/COPY용 데이터/finance_data_1999_2020_raw.csv', dtype={'stock_code':object}, chunksize=1)
get_header_finance = next(df_cache).columns

with psycopg2.connect(**connect_param) as con:
    with con.cursor() as cur:
        # 재무 데이터 테이블 만들기
        cur.execute('drop table if exists finance_data_1999_2020_raw')
        create_stocktable_sql = '''
        create table finance_data_1999_2020_raw(
        stock_code char(6),
        FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code),
        company_name varchar(50),
        결산월 smallint,
        회계년 smallint
        );
        '''
        cur.execute(create_stocktable_sql)
        con.commit()
        
        # 테이블에 열 추가하기
        prefix_add_columns = 'ALTER TABLE finance_data_1999_2020_raw ADD COLUMN'
        for each_column in get_header_finance[4:]:
            if each_column in ['부채비율(%)', '자기자본비율(%)' ,'당기순이익증가율(전년동기)(%)', 'EPS증가율(전년동기)(%)', 'ROE(당기순이익)(%)', '매출액증가율(전년동기)(%)']:
                add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' varchar(15)'
            else:
                add_columns_to_table = prefix_add_columns + ' ' + '"' + each_column + '"' + ' double precision'
            cur.execute(add_columns_to_table)
            print(each_column)
        con.commit()

총자산(천원)
유동자산(천원)
현금및현금성자산(천원)
재고자산(천원)
비유동자산(천원)
총부채(천원)
유동부채(천원)
비유동부채(천원)
총자본(천원)
매출액(천원)
당기순이익(천원)
세전계속사업이익(천원)
영업이익(천원)
이자비용(천원)
비지배주주지분(천원)
계속사업법인세비용(천원)
매출총이익(천원)
금융원가(비영업)(천원)
부채비율(%)
자기자본비율(%)
차입금의존도(%)
당기순이익률(%)
세전계속사업이익률(%)
영업이익률(%)
매출총이익률(%)
ROE(당기순이익)(%)
총자산회전율(회)
매출액증가율(전년동기)(%)
총자산증가율(전년동기)(%)
당기순이익증가율(전년동기)(%)
EPS증가율(전년동기)(%)


In [20]:
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-predict/finance_data_1999_2020_raw.csv', table_name='finance_data_1999_2020_raw')

## 4-2. 재무 데이터 머신러닝 학습용 테이블 만들기
* 3개월 후 예측
* 6개월 후 예측
* 1년 후 예측

In [11]:
# R1 : D1~D30
# R2 : D31~D60
# R3 : D61~D90
# R4 : D91~D120

step_1_column_finance_3 = ['Y-1_총자산(천원)'
,'Y-1_현금및현금성자산(천원)'
,'Y-1_총부채(천원)'
,'Y-1_총자본(천원)'
,'Y-1_매출액(천원)'
,'Y-1_당기순이익(천원)'
,'Y-1_세전계속사업이익(천원)'
,'Y-1_영업이익(천원)'
,'Y-1_매출총이익(천원)'
,'Y-1_차입금의존도(p)'
,'Y-1_당기순이익률(p)'
,'Y-1_세전계속사업이익률(p)'
,'Y-1_영업이익률(p)'
,'Y-1_매출총이익률(p)'
,'Y-1_총자산회전율(회)'
,'Y-1_총자산증가율(p)'
,'Y-1_총자산부채비율(p)'
,'Y-2_총자산(천원)'
,'Y-2_현금및현금성자산(천원)'
,'Y-2_총부채(천원)'
,'Y-2_총자본(천원)'
,'Y-2_매출액(천원)'
,'Y-2_당기순이익(천원)'
,'Y-2_세전계속사업이익(천원)'
,'Y-2_영업이익(천원)'
,'Y-2_매출총이익(천원)'
,'Y-2_차입금의존도(p)'
,'Y-2_당기순이익률(p)'
,'Y-2_세전계속사업이익률(p)'
,'Y-2_영업이익률(p)'
,'Y-2_매출총이익률(p)'
,'Y-2_총자산회전율(회)'
,'Y-2_총자산증가율(p)'
,'Y-2_총자산부채비율(p)'
,'Y-3_총자산(천원)'
,'Y-3_현금및현금성자산(천원)'
,'Y-3_총부채(천원)'
,'Y-3_총자본(천원)'
,'Y-3_매출액(천원)'
,'Y-3_당기순이익(천원)'
,'Y-3_세전계속사업이익(천원)'
,'Y-3_영업이익(천원)'
,'Y-3_매출총이익(천원)'
,'Y-3_차입금의존도(p)'
,'Y-3_당기순이익률(p)'
,'Y-3_세전계속사업이익률(p)'
,'Y-3_영업이익률(p)'
,'Y-3_매출총이익률(p)'
,'Y-3_총자산회전율(회)'
,'Y-3_총자산증가율(p)'
,'Y-3_총자산부채비율(p)']

step_1_column_finance_2 = ['Y-1_총자산(천원)'
,'Y-1_현금및현금성자산(천원)'
,'Y-1_총부채(천원)'
,'Y-1_총자본(천원)'
,'Y-1_매출액(천원)'
,'Y-1_당기순이익(천원)'
,'Y-1_세전계속사업이익(천원)'
,'Y-1_영업이익(천원)'
,'Y-1_매출총이익(천원)'
,'Y-1_차입금의존도(p)'
,'Y-1_당기순이익률(p)'
,'Y-1_세전계속사업이익률(p)'
,'Y-1_영업이익률(p)'
,'Y-1_매출총이익률(p)'
,'Y-1_총자산회전율(회)'
,'Y-1_총자산증가율(p)'
,'Y-1_총자산부채비율(p)'
,'Y-2_총자산(천원)'
,'Y-2_현금및현금성자산(천원)'
,'Y-2_총부채(천원)'
,'Y-2_총자본(천원)'
,'Y-2_매출액(천원)'
,'Y-2_당기순이익(천원)'
,'Y-2_세전계속사업이익(천원)'
,'Y-2_영업이익(천원)'
,'Y-2_매출총이익(천원)'
,'Y-2_차입금의존도(p)'
,'Y-2_당기순이익률(p)'
,'Y-2_세전계속사업이익률(p)'
,'Y-2_영업이익률(p)'
,'Y-2_매출총이익률(p)'
,'Y-2_총자산회전율(회)'
,'Y-2_총자산증가율(p)'
,'Y-2_총자산부채비율(p)'
]

extracted_value_finance = [
'총자산(천원)'
,'현금및현금성자산(천원)'
,'총부채(천원)'
,'총자본(천원)'
,'매출액(천원)'
,'당기순이익(천원)'
,'세전계속사업이익(천원)'
,'영업이익(천원)'
,'매출총이익(천원)'
,'차입금의존도(p)'
,'당기순이익률(p)'
,'세전계속사업이익률(p)'
,'영업이익률(p)'
,'매출총이익률(p)'
,'총자산회전율(회)'
,'총자산증가율(전년동기)(p)'
,'총자산부채비율(p)'  
]

In [16]:
def create_finance_table_for_ml(prediction_range='6m', years=2):
    
    table_name_argument = 'table_for_dt_based_model_' + prediction_range + '_finance_' + str(years) + 'y'
    if years == 3:
        step_1_column_finance = step_1_column_finance_3
    elif years == 2:
        step_1_column_finance = step_1_column_finance_2
    
    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:
            # 재무 데이터 테이블 만들기
            drop_query = sql.SQL('drop table if exists {table_name}').format(table_name=sql.Identifier(table_name_argument))
            cur.execute(drop_query)
            create_stocktable_sql = sql.SQL("""
            create table {table_name}(
            stock_code char(6),
            FOREIGN KEY (stock_code) REFERENCES target_company_list(stock_code)
            );
            """).format(table_name=sql.Identifier(table_name_argument))
            cur.execute(create_stocktable_sql)

            # 테이블에 열 추가하기
            for each_column in step_1_column_finance:
                query_prefix_add_columns = sql.SQL('ALTER TABLE {table_name} ADD COLUMN {column} double precision').format(table_name = sql.Identifier(table_name_argument), column = sql.Identifier(each_column))
                cur.execute(query_prefix_add_columns)
                print(each_column)
            con.commit()

In [ ]:
create_finance_table_for_ml(prediction_range='1y', years=3)
create_finance_table_for_ml(prediction_range='1y', years=2)
create_finance_table_for_ml(prediction_range='6m', years=3)
create_finance_table_for_ml(prediction_range='6m', years=2)
create_finance_table_for_ml(prediction_range='3m', years=3)
create_finance_table_for_ml(prediction_range='3m', years=2)

In [18]:
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-prediction/table_for_dt_based_model_1y_finance_3y.csv', table_name='table_for_dt_based_model_1y_finance_3y')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-prediction/table_for_dt_based_model_1y_finance_2y.csv', table_name='table_for_dt_based_model_1y_finance_2y')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-prediction/table_for_dt_based_model_6m_finance_3y.csv', table_name='table_for_dt_based_model_6m_finance_3y')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-prediction/table_for_dt_based_model_6m_finance_2y.csv', table_name='table_for_dt_based_model_6m_finance_2y')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-prediction/table_for_dt_based_model_3m_finance_3y.csv', table_name='table_for_dt_based_model_3m_finance_3y')
copy_csv_from_s3(connect_param, s3_bucket_param='s3://for-bankruptcy-prediction/table_for_dt_based_model_3m_finance_2y.csv', table_name='table_for_dt_based_model_3m_finance_2y')